In [1]:
import tensorflow as tf

DATASET_PATH = "saved_word2vec_dataset"
loaded_dataset = tf.data.Dataset.load(DATASET_PATH)
print(loaded_dataset)

2025-12-14 14:30:24.933924: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


<_LoadDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


2025-12-14 14:30:35.600328: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [2]:
dataset = loaded_dataset.unbatch()
print(dataset)


<_UnbatchDataset element_spec=((TensorSpec(shape=(), dtype=tf.int64, name=None), TensorSpec(shape=(5,), dtype=tf.int64, name=None)), TensorSpec(shape=(5,), dtype=tf.int64, name=None))>


In [4]:
import numpy as np

targets, contexts, labels = zip(*[
    (t.numpy(), c.numpy(), l.numpy())
    for (t, c), l in dataset
])

targets = np.array(targets)
contexts = np.array(contexts)
labels = np.array(labels)

2025-12-14 14:36:22.247638: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [5]:
targets.shape, contexts.shape, labels.shape

((620544,), (620544, 5), (620544, 5))

In [7]:
v = 4096
d = 100
lr = 0.025

In [8]:
rng = np.random.default_rng(42)

w_in = rng.normal(0, .01, size=(v, d))
w_out = rng.normal(0, .01, size=(v, d))

In [10]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [15]:
a = True
def sgd_step(t, ctx, lbl):
    """
    t   : int
    ctx : (k, )
    lbl : (k, )
    """

    global a
    v_t = w_in[t]
    v_c = w_out[ctx]

    if a:
        print(v_t, v_c)
    scores = np.dot(v_c, v_t)
    if a:
        print(scores)
    probs = sigmoid(scores)

    error = probs - lbl

    grad_out = error[:, None] * v_t
    grad_in = np.sum(error[:, None] * v_c, axis=0)

    w_out[ctx] -= lr * grad_out
    w_in[t] -= lr * grad_in

    loss = -np.sum(
        lbl * np.log(probs + 1e-10) +
        (1 - lbl) * np.log(1 - probs + 1e-10)
    )

    a = False
    return loss

In [16]:
def train(targets, contexts, labels, epochs=1):
    N = len(targets)

    for e in range(epochs):
        total_loss = 0
        order = np.random.permutation(N)

        for i in order:
            loss = sgd_step(targets[i], contexts[i], labels[i])
            total_loss += loss

        print(f"Epoch {e+1}: Average loss = {total_loss / N:.4f}")

In [17]:
train(
    targets, contexts, labels, epochs=3
)

[ 0.00750664  0.00035251 -0.00030586  0.00694099 -0.00057483  0.00143495
 -0.00985283  0.01205205 -0.0124552  -0.0034188  -0.00058955  0.02038235
 -0.00402257 -0.00799815 -0.00515241  0.01552322 -0.02756102 -0.01030546
  0.02864682  0.00413242 -0.01121756 -0.00015383 -0.00024073  0.00765077
  0.01889501  0.00255057 -0.00432766  0.00724643 -0.00702188 -0.01027137
 -0.01751609 -0.01506578 -0.01065056 -0.01146939  0.0034137  -0.01397053
 -0.03007406 -0.01019379 -0.00477835 -0.0059248   0.01567546 -0.00568527
  0.00920048 -0.00029476  0.00022483 -0.00906797  0.00055098 -0.00663226
  0.00274359 -0.00304419 -0.00944547  0.00952669  0.00269486 -0.00421594
  0.00088981  0.00040352 -0.01058242  0.00031621 -0.00724673 -0.00826318
 -0.01714172 -0.00334476  0.00470727 -0.00668448  0.02645161  0.0094638
  0.01217964  0.00836816  0.01851673 -0.00740028 -0.02571556 -0.00364929
  0.00487445  0.00142579  0.00260296  0.01172088  0.00287459  0.00753076
 -0.00830726 -0.01093521  0.01738736  0.00188667 -0.

In [18]:
def most_similar(word, vocab, inv_vocab, top_k=5):
    idx = vocab[word]
    v = w_in[idx]

    norms = np.linalg.norm(w_in, axis=1)
    sims = np.dot(w_in, v) / (norms * np.linalg.norm(v) + 1e-10)

    best = np.argsort(-sims)[1:top_k+1]
    return [(inv_vocab[i], float(sims[i])) for i in best]


In [19]:
import io

inverse_vocab = {}
vocab = {}

with io.open("metadata.tsv", encoding="utf-8") as f:
    for idx, word in enumerate(f):
        word = word.strip()
        inverse_vocab[idx] = word
        vocab[word] = idx


In [20]:
print(inverse_vocab[1])
print(vocab[inverse_vocab[1]])


the
1


In [21]:
def most_similar(word, top_k=5):
    idx = vocab[word]
    v = w_in[idx]

    norms = np.linalg.norm(w_in, axis=1)
    sims = np.dot(w_in, v) / (norms * np.linalg.norm(v) + 1e-10)

    best = np.argsort(-sims)[1:top_k+1]
    return [(inverse_vocab[i], float(sims[i])) for i in best]


In [22]:
most_similar("first")


[('that', 0.6861552094899261),
 ('hortensio', 0.6570540506313828),
 ('bloody', 0.6279546609489267),
 ('turns', 0.6050210358206534),
 ('general', 0.5726904082603838)]